<a href="https://colab.research.google.com/github/juansokil/Scripts-Tesis/blob/master/02.Preprocesamiento_BaseDatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# -*- coding: utf-8 -*-

import pandas as pd

# spacy for lemmatization
import spacy
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en_core_web_sm')

# Gensim
import gensim
import gensim.corpora as corpora


# Save objects
import pickle

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


Levanta los datos - Esta base es la que se terminó exportado en el punto anterior

In [0]:

base_genero = pd.read_csv("https://raw.githubusercontent.com/juansokil/Scripts-Tesis/master/bases_genero/base.csv", sep='\t', encoding='latin1')
#base_genero = pd.read_csv("C:/Users/Juan/Dropbox/POSGRADOS/Uba/99-Tesis/scripts/scripts/base_completa/base.csv", sep='\t', encoding='latin1')
base_genero = base_genero.set_index('id')


###Selecciono una muestra###
base_genero=base_genero.sample(n=100, random_state=1)


In [0]:
#pasa a minuscula
base_genero['Resumen'] = base_genero['Resumen'].str.lower()
#Largo de cada abstract
base_genero['Resumen'].str.len()

In [0]:

##Crea variable titulo + abstract
base_genero['Resumen'] = base_genero.Titulo.astype(str).str.cat(base_genero.Resumen.astype(str), sep=' ')
base_genero['Resumen'] = base_genero['Resumen'].str.lower()  



Genero las funciones para procesar el texto

In [0]:
################FUNCION DE TGOKENIZAR##########
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

################FUNCION DE LEMATIZAR##########
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

Aplico la función de Tokens

In [6]:

###############APLICO LA FUNCION DE TGOKENIZAR###################
data_words = list(sent_to_words(base_genero['Resumen']))
data_words_se = pd.Series(data_words)
#base_genero['data_words'] = data_words_se.values
print(data_words[2])


['exploring', 'university', 'studentsa', 'gender', 'role', 'attitudes', 'and', 'their', 'effects', 'on', 'sexuality', 'and', 'behaviour', 'towards', 'hiv', 'aids', 'prevention', 'case', 'study', 'the', 'paper', 'investigated', 'the', 'effects', 'of', 'gender', 'attitudes', 'and', 'the', 'extent', 'to', 'which', 'they', 'impact', 'on', 'the', 'sexuality', 'of', 'third', 'year', 'bachelor', 'degree', 'students', 'in', 'zimbabwean', 'university', 'the', 'goal', 'of', 'the', 'study', 'was', 'to', 'establish', 'their', 'level', 'of', 'awareness', 'to', 'the', 'existing', 'hiv', 'aids', 'preventive', 'measures', 'the', 'study', 'used', 'total', 'sample', 'size', 'of', 'participants', 'selected', 'from', 'four', 'faculties', 'in', 'the', 'university', 'through', 'the', 'gender', 'stratified', 'random', 'sampling', 'technique', 'the', 'design', 'adopted', 'was', 'quantitative', 'case', 'study', 'and', 'data', 'were', 'elicited', 'by', 'means', 'of', 'questionnaires', 'focus', 'group', 'intervi

Lematizar los resumenes manteniendo solo: Noun, Adj, Verb, Adverb

In [7]:

###############APLICO LA FUNCION DE LEMATIZAR###################
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized_se = pd.Series(data_lemmatized)
base_genero['data_lemmatized'] = data_lemmatized_se.values
print(data_lemmatized[2])


explore university studentsa gender role attitude effect sexuality behaviour hiv aid prevention case study paper investigate effect gender attitude extent impact sexuality third year bachelor degree student zimbabwean university goal study be establish level awareness exist hiv aid preventive measure study use total sample size participant select faculty university gender stratify random sampling technique design adopt be quantitative case study datum be elicit mean questionnaire focus group interview observation co curricular activity analysis finding follow use descriptive inferential statistic statistical model such chi square test significance be adopt establish studentsa differential gender attitude sexuality awareness existence hiv aid preventive measure major finding indicate studentsa attitude sexuality be gender base awareness hiv aid preventive strategy be mainly influence gender stereotype cultural belief femininity masculinity study conclude there be imperative need social 

Importa las Stopwords y agrega las necesarias

In [0]:
####GENERALES####
from spacy.lang.en.stop_words import STOP_WORDS

####DOMINIO####
stopwords_dominio = ['(c)', 'abstract', 'aim', 'analysis', 'analize', 'approach', 'argue', 'article', 'author', 'authors', 
                     'background', 'compare', 'conclude', 'conclusion', 'conclusions', 'context', 'copyright', 'data', 'de', 
                     'dept', 'discuss', 'discussed', 'discussion', 'elsevier', 'emerald', 'examine', 'examined', 'experience', 
                     'experienced', 'explore', 'finding', 'findings', 'find', 'focus', 'focused', 'franci', 'francis', 'group', 
                     'hypothesis',  'idea', 'impact', 'implications', 'inc', 'informa', 'information', 'inform', 'interview', 
                     'introduction', 'journal', 'la', 'limit', 'limitations', 'limited', 'literature', 'llc', 'ltd', 
                     'materials', 'measure', 'measured', 'methodology', 'method', 'methods', 'model', 'na', 'objective', 
                     'paper', 'predict', 'prediction', 'present', 'publication', 'publications', 'published', 'publishing', 
                     'purpose', 'qualitative', 'quantitative', 'related', 'report', 'reported', 'research', 'researched', 
                     'result', 'results', 'sage', 'sample', 'springer', 'studies', 'study', 'studying', 'suggest', 'survey', 
                     'taylor', 'theory', 'thesis', 'understand' 'university', 'univ']  


####PAISES####
stopwords_countries = ['Africa', 'Europe', 'Asia', 'America', 'Oceania', 'Afghanistan',  'Albania',  'Algeria',  'Andorra',  'Angola',  'Antigua and Barbuda',  'Argentina', 
                       'Armenia',  'Australia',  'Austria',  'Azerbaijan',  'Bahamas',  'Bahrain',  'Bangladesh',  'Barbados',  
                       'Belarus',  'Belgium',  'Belize',  'Benin',  'Bhutan',  'Bolivia',  'Bosnia and Herzegovina',  
                       'Botswana',  'Brazil',  'Brunei',  'Bulgaria',  'Burkina Faso',  'Burundi',  "Côte d'Ivoire",  
                       'Cabo Verde',  'Cambodia',  'Cameroon',  'Canada',  'Central African Republic',  'Chad',  'Chile',  
                       'China',  'Colombia',  'Comoros',  'Congo (Congo-Brazzaville)',  'Congo Republic', 'Costa Rica',  'Croatia',  'Cuba',  'Cyprus',  'Czechia',  'Democratic Republic of the Congo',  'Denmark',  'Djibouti',  'Dominica',  'Dominican Republic',  'Ecuador',  'Egypt',  'El Salvador',  'Equatorial Guinea',  'Eritrea',  'Estonia',  'Ethiopia',  'Fiji',  'Finland',  'France',  'Gabon',  'Gambia',  'Georgia',  'Germany',  'Ghana',  'Greece',  'Grenada',  'Guatemala',  'Guinea',  'Guinea-Bissau',  'Guyana',  'Haiti',  'Holy See',  'Honduras',  'Hungary',  'Iceland',  'India',  'Indonesia',  'Iran',  'Iraq',  'Ireland',  'Israel',  'Italy',  'Jamaica',  'Japan',  'Jordan',  'Kazakhstan',  'Kenya',  'Kiribati',  'Kuwait',  'Kyrgyzstan',  'Laos',  'Latvia',  'Lebanon',  'Lesotho',  'Liberia',  'Libya',  'Liechtenstein',  'Lithuania',  'Luxembourg',  'Madagascar',  'Malawi',  'Malaysia',  'Maldives',  'Mali',  'Malta',  'Marshall Islands',  'Mauritania',  'Mauritius',  'Mexico',  'Micronesia',  'Moldova',  'Monaco',  'Mongolia',  'Montenegro',  'Morocco',  'Mozambique',  'Myanmar (formerly Burma)',  'Namibia',  'Nauru',  'Nepal',  'Netherlands',  'New Zealand',  'Nicaragua',  'Niger',  'Nigeria',  'North Korea',  'North Macedonia',  'Norway',  'Oman',  'Pakistan',  'Palau',  'Palestine State',  'Panama',  'Papua New Guinea',  'Paraguay',  'Peru',  'Philippines',  'Poland',  'Portugal',  'Qatar',  'Romania',  'Russia',  'Rwanda',  'Saint Kitts and Nevis',  'Saint Lucia',  'Saint Vincent and the Grenadines',  'Samoa',  'San Marino',  'Sao Tome and Principe',  'Saudi Arabia',  'Senegal',  'Serbia',  'Seychelles',  'Sierra Leone',  'Singapore',  'Slovakia',  'Slovenia',  'Solomon Islands',  'Somalia',  'South Africa',  'South Korea',  'South Sudan',  'Spain',  'Sri Lanka',  'Sudan',  'Suriname',  'Swaziland',  'Sweden',  'Switzerland',  'Syria',  'Tajikistan',  'Tanzania',  'Thailand',  'Timor-Leste',  'Togo',  'Tonga',  'Trinidad and Tobago',  'Tunisia',  'Turkey',  'Turkmenistan',  'Tuvalu',  'Uganda',  'Ukraine',  'United Arab Emirates',  'United Kingdom',  'United States of America',  'Uruguay',  'Uzbekistan',  'Vanuatu',  'Venezuela',  'Vietnam',  'Yemen',  'Zambia',  'Zimbabwe']  


for word in stopwords_dominio:
    STOP_WORDS.add(word)

for word in stopwords_countries:
    STOP_WORDS.add(word)
    
    
base_genero['data_lemmatized_stopwords'] = base_genero['data_lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word not in (STOP_WORDS)]))


In [9]:
base_genero.head()

,Año,Revista,Titulo,Resumen,Tipo,Pais,data_lemmatized,data_lemmatized_stopwords
id,,,,,,,,
2-s2.0-85021819243,2017,Asian Women,Advocating policy interests in local politics:...,advocating policy interests in local politics:...,Article,Japan,advocate policy interest local politic woman s...,advocate policy interest local politic woman s...
2-s2.0-84855744244,2012,International Journal of Entrepreneurial Behav...,Gender and business ownership: Questioning â...,gender and business ownership: questioning â...,Article,United States,gender business ownership question whata whya ...,gender business ownership question whata whya ...
2-s2.0-84917689712,2014,Mediterranean Journal of Social Sciences,Exploring university studentsâ gender role a...,exploring university studentsâ gender role a...,Article,South Africa,explore university studentsa gender role attit...,university studentsa gender role attitude effe...
2-s2.0-84947927456,2015,Comparative Social Research,Two sides of the same coin? Applied and genera...,two sides of the same coin? applied and genera...,Article,Canada,side same coin apply general high education ge...,coin apply general high education gender strat...
2-s2.0-33644790108,2006,Violence and Victims,Influence of gender and experience on college ...,influence of gender and experience on college ...,Article,United States,influence gender experience college student st...,influence gender college student stalk schema ...


Ya tengo el campo de texto preprocesado, ahora vuelvo a ejecutar el procedimiento final

In [0]:

######################EN ESTE PASO VUELVO A HACER LA TOKENIZACION CON LOS DATOS FINALES#########
data_words = list(sent_to_words(base_genero['data_lemmatized_stopwords']))


###################### CONSTRUCCION DE N-GRAMAS#######
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=0.1) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=0.075)  
cuatrigram = gensim.models.Phrases(trigram[bigram[data_words]], threshold=0.05)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
cuatrigram_mod = gensim.models.phrases.Phraser(cuatrigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def make_cuatrigrams(texts):
    return [cuatrigram_mod[trigram_mod[bigram_mod[doc]]] for doc in texts]


  # Form Trigrams
make_cuatrigrams = make_cuatrigrams(data_words)


Puedo listar uno de los casos para ver las palabras que tiene:

In [11]:
print('Cantidad de palabras: ', len(make_cuatrigrams[2]))
print(make_cuatrigrams[2])

Cantidad de palabras:  127
['university', 'studentsa', 'gender', 'role', 'attitude', 'effect', 'sexuality', 'behaviour', 'hiv', 'aid', 'prevention', 'case', 'investigate', 'effect_gender', 'attitude', 'extent', 'sexuality', 'year', 'bachelor', 'degree', 'student', 'zimbabwean', 'university', 'goal', 'establish', 'level', 'awareness', 'exist', 'hiv', 'aid', 'preventive', 'use', 'total', 'size', 'participant', 'select', 'faculty', 'university', 'gender', 'stratify', 'random', 'sampling', 'technique', 'design', 'adopt', 'case', 'datum', 'elicit', 'mean', 'questionnaire', 'observation', 'co', 'curricular', 'activity', 'follow', 'use', 'descriptive', 'inferential', 'statistic', 'statistical', 'chi', 'square', 'test', 'significance', 'adopt', 'establish', 'studentsa', 'differential', 'gender', 'attitude', 'sexuality', 'awareness', 'existence', 'hiv', 'aid', 'preventive', 'major', 'indicate', 'studentsa', 'attitude', 'sexuality', 'gender', 'base', 'awareness', 'hiv', 'aid', 'preventive', 'str

Elimina los tokens generales que aparecen en mas del 20% de las publicaciones - Esto es una decisión teórica para no ensuciar los resultados posteriores

In [0]:
for i in range(len(make_cuatrigrams)):
    for word in make_cuatrigrams[i]:
        if 'gender' in make_cuatrigrams[i]:
            #print('Si')
            make_cuatrigrams[i].remove('gender')
        if 'woman' in make_cuatrigrams[i]:
            #print('Si')
            make_cuatrigrams[i].remove('woman')
        if 'female' in make_cuatrigrams[i]:
            #print('Si')
            make_cuatrigrams[i].remove('female')
        if 'social' in make_cuatrigrams[i]:
            #print('Si')
            make_cuatrigrams[i].remove('social')
        if 'difference' in make_cuatrigrams[i]:
            #print('Si')
            make_cuatrigrams[i].remove('difference')
            #print('Despues: ',len(make_cuatrigrams[i]))

Una vez que limpio las palabras muy frecuentes puedo ver las diferencias

In [13]:
print('Cantidad de palabras: ', len(make_cuatrigrams[2]))
print(make_cuatrigrams[2])

Cantidad de palabras:  117
['university', 'studentsa', 'role', 'attitude', 'effect', 'sexuality', 'behaviour', 'hiv', 'aid', 'prevention', 'case', 'investigate', 'effect_gender', 'attitude', 'extent', 'sexuality', 'year', 'bachelor', 'degree', 'student', 'zimbabwean', 'university', 'goal', 'establish', 'level', 'awareness', 'exist', 'hiv', 'aid', 'preventive', 'use', 'total', 'size', 'participant', 'select', 'faculty', 'university', 'stratify', 'random', 'sampling', 'technique', 'design', 'adopt', 'case', 'datum', 'elicit', 'mean', 'questionnaire', 'observation', 'co', 'curricular', 'activity', 'follow', 'use', 'descriptive', 'inferential', 'statistic', 'statistical', 'chi', 'square', 'test', 'significance', 'adopt', 'establish', 'studentsa', 'differential', 'attitude', 'sexuality', 'awareness', 'existence', 'hiv', 'aid', 'preventive', 'major', 'indicate', 'studentsa', 'attitude', 'sexuality', 'base', 'awareness', 'hiv', 'aid', 'preventive', 'strategy', 'mainly', 'influence', 'stereoty

In [0]:
### Creo una variable que se llama n-gramas
make_cuatrigrams_se = pd.Series(make_cuatrigrams)
base_genero['ngramas'] = make_cuatrigrams_se.values

Miro el diccionario

In [15]:
##################################PASOS DICCIONARIO########################
# Create Dictionary
id2word = corpora.Dictionary(base_genero['ngramas'])
print('Diccionario Inicial: ', len(id2word))

id2word.filter_extremes(no_below=5, no_above=0.2)
id2word.compactify()
print('Diccionario Filtrado: ', len(id2word))


# Create Corpus
texts = base_genero['ngramas']
# Term Document Frequency - Diccionario
bow = [id2word.doc2bow(text) for text in texts]

# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in bow[2:3]]

Diccionario Inicial:  2630
Diccionario Filtrado:  290


[[('case', 2),
  ('exist', 1),
  ('investigate', 1),
  ('effect_gender', 1),
  ('indicate', 1),
  ('activity', 2),
  ('adopt', 2),
  ('attitude', 4),
  ('base', 1),
  ('cultural', 1),
  ('degree', 1),
  ('design', 1),
  ('discourse', 1),
  ('establish', 2),
  ('follow', 1),
  ('girl', 1),
  ('influence', 1),
  ('level', 1),
  ('masculinity', 1),
  ('need', 1),
  ('observation', 1),
  ('parent', 1),
  ('participant', 1),
  ('questionnaire', 1),
  ('right_reserve', 1),
  ('select', 1),
  ('sexuality', 4),
  ('society', 1),
  ('stereotype', 2),
  ('strategy', 1),
  ('structure', 1),
  ('student', 1),
  ('test', 1),
  ('total', 1),
  ('university', 3),
  ('year', 1)]]

Guardo los objetos

In [0]:
###Guardar datos####
#id2word.save('../LDA/dictionary.pkl')
pickle.dump(id2word, open("./dictionary.pkl", "wb"))
pickle.dump(bow, open('./corpus.pkl', 'wb'))

Los abro para controlar que esten bien

In [0]:
###Carga diccionario (y chequea) ###
dictionary = pickle.load(open(".dictionary.pkl", "rb"))
corpus = pickle.load(open(".corpus.pkl", "rb"))

